
# 对大数据进行预处理

以占领华尔街推特数据为例



![image.png](./images/author.png)

## 字节（Byte /bait/）

计算机信息技术用于计量存储容量的一种计量单位，通常情况下一字节等于有八位， [1]  也表示一些计算机编程语言中的数据类型和语言字符。
- 1B（byte，字节）= 8 bit；
- 1KB=1000B；1MB=1000KB=1000×1000B。其中1000=10^3。
- 1KB（kilobyte，千字节）=1000B= 10^3 B；
- 1MB（Megabyte，兆字节，百万字节，简称“兆”）=1000KB= 10^6 B；
- 1GB（Gigabyte，吉字节，十亿字节，又称“千兆”）=1000MB= 10^9 B；

## 分段读取数据并进行处理

Lazy Method for Reading Big File in Python?

In [1]:
# import sys

# flush print
# def flushPrint(d):
#     sys.stdout.write('\r')
#     sys.stdout.write(str(d))
#     sys.stdout.flush()
for i in range(10):    
    print(i, end= '\r')

In [2]:
# 按行读取数据
line_num = 0
cops_num = 0
# windows users may need to add encoding = 'utf8' into the folling line.
with open('/Users/datalab/bigdata/cjc/ows-raw.txt', 'r') as f:
    for i in f:
        line_num += 1
        if 'cops' in i:
            cops_num += 1
        if line_num % 100000 ==0:
            print(line_num, end='\r')

In [3]:
line_num

6911408

In [5]:
cops_num/line_num

0.011413448605551865

In [6]:
bigfile = open('/Users/datalab/bigdata/cjc/ows-raw.txt', 'r')
chunkSize = 1000000
chunk = bigfile.readlines(chunkSize)
print(len(chunk))
# with open("../data/ows_tweets_sample.txt", 'w') as f:
#     for i in chunk:
#         f.write(i)  

2754


In [5]:
bigfile.readlines?

In [8]:
5%5

0

In [7]:
# https://stackoverflow.com/questions/519633/lazy-method-for-reading-big-file-in-python?lq=1
import csv
bigfile = open('/Users/datalab/bigdata/cjc/ows-raw.txt', 'r')
chunkSize = 10**8
chunk = bigfile.readlines(chunkSize)
num_chunk, num_lines, num_cops = 0, 0, 0
while chunk:
    lines = csv.reader((line.replace('\x00','') for line in chunk), 
                       delimiter=',', quotechar='"')
    # do sth.
    num_lines += len(list(lines))
#     for i in lines:
#         if 'cops' in i:
#             num_cops +=1
    if num_chunk % 5 ==0:
        print(num_chunk, num_lines, end = '\r')
    num_chunk += 1
    chunk = bigfile.readlines(chunkSize) # read another chunk

In [16]:
num_lines

6602141

## 用Pandas的get_chunk功能来处理亿级数据

> 只有在超过5TB数据量的规模下，Hadoop才是一个合理的技术选择。

In [11]:
import pandas as pd

f = open('/Users/datalab/bigdata/cjc/ows-raw.txt',encoding='utf-8')
reader = pd.read_table(f,  sep=',',  quotechar='"', iterator=True, on_bad_lines='skip') #跳过报错行
chunkSize = 100000
chunk = reader.get_chunk(chunkSize)
len(chunk)

#pd.read_table?

100000

In [15]:
chunk.head()

,Twitter ID,Text,Profile Image URL,Day,Hour,Minute,Created At,Geo,From User,From User ID,Language,To User,To User ID,Source
0,121813144174727168,RT @AnonKitsu: ALERT!!!!!!!!!!COPS ARE KETTLIN...,http://a2.twimg.com/profile_images/1539375713/...,2011-10-06,5,4,2011-10-06 05:04:51,N;,Anonops_Cop,401240477,en,NaN,0,&lt;a href=&quot;http://twitter.com/&quot;&gt;...
1,121813146137657344,@jamiekilstein @allisonkilkenny Interesting in...,http://a2.twimg.com/profile_images/1574715503/...,2011-10-06,5,4,2011-10-06 05:04:51,N;,KittyHybrid,34532053,en,jamiekilstein,2149053,&lt;a href=&quot;http://twitter.com/&quot;&gt;...
2,121813150000619521,@Seductivpancake Right! Those guys have a vict...,http://a1.twimg.com/profile_images/1241412831/...,2011-10-06,5,4,2011-10-06 05:04:52,N;,nerdsherpa,95067344,en,Seductivpancake,19695580,&lt;a href=&quot;http://www.echofon.com/&quot;...
3,121813150701072385,RT @bembel &quot;Occupy Wall Street&quot; als ...,http://a0.twimg.com/profile_images/1106399092/...,2011-10-06,5,4,2011-10-06 05:04:52,N;,hamudistan,35862923,en,NaN,0,&lt;a href=&quot;http://levelupstudio.com&quot...
4,121813163778899968,#ows White shirt= Brown shirt.,http://a2.twimg.com/profile_images/1568117871/...,2011-10-06,5,4,2011-10-06 05:04:56,N;,kl_knox,419580636,en,NaN,0,&lt;a href=&quot;http://twitter.com/&quot;&gt;...


In [51]:
import pandas as pd

f = open('/Users/datalab/bigdata/cjc/ows-raw.txt',encoding='utf-8')
reader = pd.read_table(f,  sep=',',  quotechar='"', 
                       iterator=True, on_bad_lines='skip') #跳过报错行
chunkSize = 100000
loop = True
cops_data = []
num_chunk, num_lines = 0, 0
while loop:
    try:
        chunk = reader.get_chunk(chunkSize)
        # dat = data_cleaning_funtion(chunk) # do sth. e.g., if cops in dat
        dat=[chunk.loc[k] for k in chunk.index if 'cops' in str(chunk['Text'][k]) ]
        num_lines += len(chunk)
        print(num_chunk, num_lines, end = '\r')
        num_chunk +=1
        for d in dat:
            cops_data.append(d) 
    except StopIteration:
        loop = False
        print("Iteration is stopped.")
#df = pd.concat(data, ignore_index=True)

Iteration is stopped.


In [47]:
chunk.loc[k]

Twitter ID                                          153352746249101312
Text                                                               NaN
Profile Image URL    http://a3.twimg.com/profile_images/1647201670/...
Day                                                         2012-01-01
Hour                                                                 5
Minute                                                              51
Created At                                         2012-01-01 05:51:58
Geo                                                                 N;
From User                                              AngeredPopulist
From User ID                                                 416481838
Language                                                            en
To User                                                            NaN
To User ID                                                           0
Source               &lt;a href=&quot;http://twitter.com/#!/downloa...
Name: 

In [55]:
len(cops_data)

78397

In [58]:
pd.concat(dat, ignore_index=True)

0                                    170734732877893632
1     RT @DiceyTroop: When I got here, cops were has...
2     http://a2.twimg.com/profile_images/1753747297/...
3                                            2012-02-18
4                                                     5
5                                                     1
6                                   2012-02-18 05:01:47
7                                                    N;
8                                             shushugah
9                                              28624302
10                                                   en
11                                                  NaN
12                                                    0
13    &lt;a href=&quot;http://twitter.com/#!/downloa...
Name: 6600282, dtype: object

In [54]:
df = pd.DataFrame.from_dict(cops_data)
df.head()


,Twitter ID,Text,Profile Image URL,Day,Hour,Minute,Created At,Geo,From User,From User ID,Language,To User,To User ID,Source
57,121813549478707200,RT @kittylight: Dear #cops THE WHOLE WORLD IS ...,http://a2.twimg.com/profile_images/1146887237/...,2011-10-06,5,6,2011-10-06 05:06:28,N;,dove_hawk,361839281,en,NaN,0,&lt;a href=&quot;http://twitter.com/#!/downloa...
95,121813722099482624,"The whiny, sanctimonious drivel coming out of ...",http://a3.twimg.com/profile_images/1573938172/...,2011-10-06,5,7,2011-10-06 05:07:09,N;,wryson,351681669,en,NaN,0,&lt;a href=&quot;http://twitter.com/&quot;&gt;...
98,121813748003508224,RT @KeithOlbermann: Again NYPD supervisors do ...,http://a1.twimg.com/profile_images/509909348/t...,2011-10-06,5,7,2011-10-06 05:07:15,N;,dannydoodar,76258793,en,NaN,0,&lt;a href=&quot;http://stone.com/Twittelator&...
267,121814376234754049,RT @kittylight: #isad #stayhungry #ThinkDiffer...,http://a2.twimg.com/profile_images/1540184395/...,2011-10-06,5,9,2011-10-06 05:09:45,N;,kittylightsCat,406361898,en,NaN,0,&lt;a href=&quot;http://twitter.com/#!/downloa...
278,121814402025533440,RT @kittylight: Dear #cops THE WHOLE WORLD IS ...,http://a2.twimg.com/profile_images/1540184395/...,2011-10-06,5,9,2011-10-06 05:09:51,N;,kittylightsCat,406361898,en,NaN,0,&lt;a href=&quot;http://twitter.com/#!/downloa...


![image.png](./images/end.png)